In [13]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../../src/')
from train import load_data
# from diffusion import DiffusionModel
# from evaluate import get_results
from omegaconf import OmegaConf
# from main import load_data, make_model
import numpy as np
import os
import glob
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
import torch
from model2 import Autoencoder, Preprocessor
import magic
import torch
wandb.login()
api = wandb.Api()

In [14]:
from procrustes import Procrustes

In [15]:
from plotly3d.plot import scatter

In [16]:
entity = "xingzhis"
project = "dmae"
sweep_id = '48pbdfl3'
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
# Initialize an empty list to store run data
runs_data = []

# Iterate through each run in the sweep
for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)

In [17]:
df['data.name']

0             hemisphere
1             hemisphere
2             hemisphere
3     hemisphere_hi_dim2
4             hemisphere
5     hemisphere_hi_dim2
6     hemisphere_hi_dim2
7     hemisphere_hi_dim2
8       make_mix_surface
9       make_mix_surface
10      make_mix_surface
11      make_mix_surface
12         eb_subset_all
13         eb_subset_all
14         eb_subset_all
15         eb_subset_all
Name: data.name, dtype: object

In [18]:
df.columns

Index(['_runtime', 'trainer/global_step', 'validation/dist_loss',
       'validation/loss', '_step', 'epoch', '_timestamp',
       'train/dist_loss_step', 'train/dist_loss_epoch',
       'train/reconstr_loss_epoch', 'train/loss_step', 'train/loss_epoch',
       'train/reconstr_loss_step', 'validation/reconstr_loss', 'data', 'loss',
       'path', 'logger', 'decoder', 'encoder', 'training', 'data.name',
       'data.root', 'dimensions', 'preprocessor', 'cfg/data/name',
       'cfg/data/root', 'cfg/path/root', 'training.mode', 'cfg/path/model',
       'cfg/logger/tags', 'cfg/training/lr', 'decoder.dropout',
       'encoder.dropout', 'cfg/data/filetype', 'cfg/logger/entity',
       'cfg/training/mode', 'cfg/training/seed', 'dimensions.latent',
       'loss.weights.dist', 'cfg/logger/project', 'decoder.batch_norm',
       'encoder.batch_norm', 'loss.weights.cycle', 'cfg/decoder/dropout',
       'cfg/dimensions/data', 'cfg/encoder/dropout', 'loss.dist_mse_decay',
       'training.max_epochs

In [19]:
df['data.name'].value_counts()

data.name
hemisphere            4
hemisphere_hi_dim2    4
make_mix_surface      4
eb_subset_all         4
Name: count, dtype: int64

In [20]:
df[(df['data.name'] == 'hemisphere_hi_dim2') & (df['training.mode'] == 'end2end')].sort_values(['validation/dist_loss', 'validation/reconstr_loss'])[['validation/loss', 'validation/dist_loss', 'validation/reconstr_loss', 'run_id', 'cfg/encoder/dropout', 'cfg/loss/dist_mse_decay', 'cfg/loss/weights/cycle', 'cfg/dimensions/latent', 'cfg/data/root']]

,validation/loss,validation/dist_loss,validation/reconstr_loss,run_id,cfg/encoder/dropout,cfg/loss/dist_mse_decay,cfg/loss/weights/cycle,cfg/dimensions/latent,cfg/data/root
6,27.275442,0.184509,40.607616,q3esf815,0,0.5,0,3,../data/negative_sampling/hifreq1/
7,33.043221,0.240987,44.971245,za021egx,0,0.5,0,2,../data/negative_sampling/hifreq1/
3,39.273033,0.266603,58.243565,29eq24yq,0,0.5,0,3,../data/negative_sampling/hifreq1noadd/
5,44.412922,0.313134,63.050991,n35f84tr,0,0.5,0,2,../data/negative_sampling/hifreq1noadd/


In [21]:
run_id = df[(df['data.name'] == 'hemisphere_hi_dim2') & (df['training.mode'] == 'end2end') & (df['cfg/dimensions/latent'] == 3) & (df['cfg/data/root'] == '../data/negative_sampling/hifreq1noadd/')].sort_values(['validation/dist_loss', 'validation/reconstr_loss'])
assert len(run_id) == 1
run_id = run_id['run_id'].values[0]

In [22]:
run = api.run(f"{entity}/{project}/{run_id}")
folder_path = '../../src/wandb/'
cfg = OmegaConf.create(run.config)
folder_list = glob.glob(f"{folder_path}*{run.id}*")
ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
ckpt_path = ckpt_files[0]
# data_path2 = os.path.join(cfg., cfg.data.name + cfg.data.filetype)
# data = np.load(data_path2, allow_pickle=True)
# x_tensor = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
# x_tensor_normalized = model.normalize(x_tensor)
cfg.data.root = '../' + cfg.data.root
# trainloader, valloader, X, phate_coords, colors, preprocessor = load_data(cfg)
model = Autoencoder.load_from_checkpoint(ckpt_path)
# cfg.dimensions.data = X.shape[1]
# cfg.training.mode = 'end2end'
# model = Autoencoder(cfg, preprocessor)
# model = Autoencoder.load_from_checkpoint(ckpt_path, preprocessor=preprocessor)
data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)
with torch.no_grad():
    model.eval()
    x = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
    z = model.encoder(x)
    xh = model.decoder(z)
zc = z.cpu().numpy()
scatter(zc, data['mask_x'], s=2, alpha=0.2)


/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning:

Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.



In [23]:
run_id = df[(df['data.name'] == 'eb_subset_all') & (df['training.mode'] == 'end2end') & (df['cfg/dimensions/latent'] == 3) & (df['cfg/data/root'] == '../data/negative_sampling/hifreq1noadd/')].sort_values(['validation/dist_loss', 'validation/reconstr_loss'])
assert len(run_id) == 1
run_id = run_id['run_id'].values[0]

In [24]:
# run_id = 'ukca8aig'
# run_id = 'ia1dnvxg'
run = api.run(f"{entity}/{project}/{run_id}")
folder_path = '../../src/wandb/'
cfg = OmegaConf.create(run.config)
folder_list = glob.glob(f"{folder_path}*{run.id}*")
ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
ckpt_path = ckpt_files[0]
# data_path2 = os.path.join(cfg., cfg.data.name + cfg.data.filetype)
# data = np.load(data_path2, allow_pickle=True)
# x_tensor = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
# x_tensor_normalized = model.normalize(x_tensor)
cfg.data.root = '../' + cfg.data.root
# trainloader, valloader, X, phate_coords, colors, preprocessor = load_data(cfg)
model = Autoencoder.load_from_checkpoint(ckpt_path)
# cfg.dimensions.data = X.shape[1]
# cfg.training.mode = 'end2end'
# model = Autoencoder(cfg, preprocessor)
# model = Autoencoder.load_from_checkpoint(ckpt_path, preprocessor=preprocessor)
data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)
with torch.no_grad():
    model.eval()
    x = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
    z = model.encoder(x)
    xh = model.decoder(z)
zc = z.cpu().numpy()
scatter(zc, data['mask_x'], s=2, alpha=0.2)
